# Reproduce DL
## Automated Pavement Crack Segmentation

We start by setting up the actual Architecture. This means making sure all weights are properly initialized and all layers are connected. 

We make use of PyTorch for the implementation.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
from architecture import main

network = main.Net()
# print(network)
summary(network, input_size=(3, 320, 480))

ModuleNotFoundError: No module named 'resnet34'

In [ ]:
# Sjoerd
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.abspath("")), '..'))


from work.CFD.data import CFD
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import torch


totensor = transforms.ToTensor()
dataset = CFD("../CFD/cfd_image", totensor, "../CFD/seg_gt", totensor)
gen = torch.Generator().manual_seed(42)
train_data, test_data = random_split(dataset, [71, 47], gen)

train_loader = DataLoader(train_data, batch_size=10)
test_loader = DataLoader(test_data, batch_size=47)


# fig, axs = plt.subplots(5, 5, figsize=(5, 5))
# for i in range(25):
#     _, gt = test_data[i]
#     ax = axs[i // 5][i % 5]
#     ax.imshow(gt.view(320, 480), cmap="gray")
#     ax.axis("off")
# plt.tight_layout()
# plt.show()



In [ ]:
def batch_dice_loss(true_val, pred_val, epsilon = 1e-8):
    """
    Dice coefficient loss
    ---
    Equivalent to F1 score. Often used when there is
    imbalance in dataset (non-crack pixels outnumber 
    crack pixels by 65:1). 
    
    Args:
        true_val: a tensor of shape [N, 1, H, W]
        predicted_val: a tensor of shape [N, 1, H, W]
    Returns:
        dice_loss: the Dice loss.
    """
    # Sigmoid -> [0, 1], reflect probabilities
    pred_val = (torch.sigmoid(pred_val) >= 0.5).float()

    # Flattened from [N, 1, H, W] to [N, H*W]
    true_val = true_val.flatten(start_dim=1)
    pred_val = pred_val.flatten(start_dim=1)

    numerator = 2. * (pred_val * true_val).sum(dim=1)
    denominator = (pred_val).sum(dim=1) + (true_val).sum(dim=1)

    return torch.mean(1 - ((numerator + epsilon) / (denominator + epsilon)))


# def dice_loss(true_val, pred_val, epsilon = 1e-8):
#     """
#     Dice coefficient loss
#     ---
#     Equivalent to F1 score. Often used when there is
#     imbalance in dataset (non-crack pixels outnumber 
#     crack pixels by 65:1). 
    
#     Args:
#         true_val: a tensor of shape [N, 1, H, W]
#         predicted_val: a tensor of shape [N, 1, H, W]
#     Returns:
#         dice_loss: the Dice loss.
#     """
#     # Sigmoid -> [0, 1], reflect probabilities
#     pred_val = (torch.sigmoid(pred_val) >= 0.5).float()

#     # Flattened from [H, W] to [H*W]
#     true_val = true_val.flatten()
#     pred_val = pred_val.flatten()

#     numerator = 2. * (pred_val * true_val).sum()
#     denominator = (pred_val).sum() + (true_val).sum()

#     return torch.mean(1 - ((numerator + epsilon) / (denominator + epsilon)))


# prediction = torch.randint(low=-255, high=256, size=(3, 3), dtype=torch.float)


# pred_is_truth = (prediction >= 0).float()
# pred_is_false = (prediction < 0).float()
# pred_is_half = torch.flip(pred_is_truth, [0, 1])

# print(pred_is_truth, "\n\n", pred_is_false, "\n\n", pred_is_half)

# print(f"different tensors should output 1: {dice_loss(pred_is_false, prediction)}")
# print(f"the opposite tensors should output 0: {dice_loss(pred_is_truth, prediction)}")
# print(f"the avg tensors should output [0,1]: {dice_loss(pred_is_half, prediction)}")

In [ ]:
# Some functions used during training

def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device


def evaluate_accuracy(data_loader, net, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    #init
    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n


In [ ]:
# training parameters
lr = 1e-3
epochs = 1

#Initialize network
net = NetBlock()
optimizer = torch.optim.AdamW(net.parameters(), lr=lr, betas=(0.9, 0.999))
criterion = batch_dice_loss

# Define list to store losses and performances of each interation
train_losses = []
train_accs = []
test_accs = []

# Try using gpu instead of cpu
device = try_gpu()

for epoch in range(epochs):

    # Network in training mode and to device
    net.train()
    net.to(device)

    # Training loop
    for i, (x_batch, y_batch) in enumerate(train_loader):

         # Set to same device
         x_batch, y_batch = x_batch.to(device), y_batch.to(device)

         # Set the gradients to zero
         optimizer.zero_grad()

         # Perform forward pass
         y_pred = net(x_batch)

         # Compute the loss
         loss = criterion(y_pred, y_batch)
         train_losses.append(loss)

         # Backward computation and update
         loss.backward()
         optimizer.step()
    
    # Compute train and test error
    train_acc = 100*evaluate_accuracy(train_loader, net.to('cpu'))
    test_acc = 100*evaluate_accuracy(test_loader, net.to('cpu'))
    
    # Development of performance
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    # Print performance
    print('Epoch: {:.0f}'.format(epoch+1))
    print('Accuracy of train set: {:.00f}%'.format(train_acc))
    print('Accuracy of test set: {:.00f}%'.format(test_acc))
    print('')


NameError: name 'NetBlock' is not defined

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [3]:
# from torchviz import make_dot

# x = torch.randn(1,3, 320, 480)
# y = net(x) 

# make_dot(y).view()

FileNotFoundError: [Errno 2] No such file or directory: 'xdg-open'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=411d58e9-cb4b-4924-bef0-2f383eff0187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>